In [99]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid')
plt.rcParams['font.size'] = 10
pd.set_option("display.max.columns", None)

In [100]:
# Load data
df = pd.read_csv("/Users/pascal/OneDrive - Universität Zürich UZH/01_Uni/6. Semester/01_Machine Learning/04_Group Project/GroupProjectDataSet.csv", sep=',')
print('Shape of data frame:', df.shape)


Shape of data frame: (1460, 81)


In [101]:
#Drop the NaN of MasVnrType because it's only 8 lines and we cannot assume NaN to be None
df = df.dropna(subset=['MasVnrType'])

In [102]:
#Convert all numerical & categorical data into strings 
df[["MSSubClass","OverallQual","OverallCond","GarageYrBlt"]] = df[["MSSubClass","OverallQual","OverallCond","GarageYrBlt"]].astype(str)

#Detect all columns with strings
string_columns = df.select_dtypes(include='object').columns


In [103]:
#I think a working example 

from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

# Create a sample DataFrame with multiple categorical columns
#data = {'color': ['red', 'green', 'blue', 'red'], 'shape': ['circle', 'square', 'circle', 'triangle'], 'size': [1, 2, 3, 4]}
#df = pd.DataFrame(data)

# Define a column transformer to one-hot encode all categorical columns
preprocessor = make_column_transformer(
    (OneHotEncoder(sparse=False), lambda x: x.select_dtypes(include='object').columns),
    remainder='passthrough'
)

# Create a pipeline with the preprocessor
pipe = Pipeline(steps=[('preprocessor', preprocessor)])

# Fit and transform the data using the pipeline
df_encoded = pipe.fit_transform(df)

# Convert the transformed data back into a DataFrame
df_encoded = pd.DataFrame(df_encoded)

# Set the column names automatically using get_feature_names_out
df_encoded.columns = preprocessor.get_feature_names_out()

print(df_encoded)

df=df_encoded


      onehotencoder__MSSubClass_120.0  onehotencoder__MSSubClass_160.0  \
0                                 0.0                              0.0   
1                                 0.0                              0.0   
2                                 0.0                              0.0   
3                                 0.0                              1.0   
4                                 0.0                              0.0   
...                               ...                              ...   
1447                              0.0                              0.0   
1448                              0.0                              0.0   
1449                              0.0                              0.0   
1450                              1.0                              0.0   
1451                              0.0                              0.0   

      onehotencoder__MSSubClass_180.0  onehotencoder__MSSubClass_190.0  \
0                                 0.0

In [104]:
#Show the amounts of NaN's 
print(df.isna().sum())
print(df.isna().sum().sum())

onehotencoder__MSSubClass_120.0    0
onehotencoder__MSSubClass_160.0    0
onehotencoder__MSSubClass_180.0    0
onehotencoder__MSSubClass_190.0    0
onehotencoder__MSSubClass_20.0     0
                                  ..
remainder__PoolArea                0
remainder__MiscVal                 0
remainder__MoSold                  0
remainder__YrSold                  0
remainder__Class                   0
Length: 433, dtype: int64
257


In [105]:
df.columns[df.isna().any()]

Index(['remainder__LotFrontage'], dtype='object')

In [106]:
#Replace all LotFrontage NaN's with 0, because if there is no connection to the road the length will be 0 
df[["remainder__LotFrontage"]] = df[["remainder__LotFrontage"]].fillna(0)
df.isna().sum().sum()

0

In [107]:
print(df_encoded.shape)
df.head(20)

(1452, 433)


,onehotencoder__MSSubClass_120.0,onehotencoder__MSSubClass_160.0,onehotencoder__MSSubClass_180.0,onehotencoder__MSSubClass_190.0,onehotencoder__MSSubClass_20.0,onehotencoder__MSSubClass_30.0,onehotencoder__MSSubClass_40.0,onehotencoder__MSSubClass_45.0,onehotencoder__MSSubClass_50.0,onehotencoder__MSSubClass_60.0,onehotencoder__MSSubClass_70.0,onehotencoder__MSSubClass_75.0,onehotencoder__MSSubClass_80.0,onehotencoder__MSSubClass_85.0,onehotencoder__MSSubClass_90.0,onehotencoder__MSZoning_C (all),onehotencoder__MSZoning_FV,onehotencoder__MSZoning_RH,onehotencoder__MSZoning_RL,onehotencoder__MSZoning_RM,onehotencoder__Street_Grvl,onehotencoder__Street_Pave,onehotencoder__Alley_Grvl,onehotencoder__Alley_Pave,onehotencoder__Alley_nan,onehotencoder__LotShape_IR1,onehotencoder__LotShape_IR2,onehotencoder__LotShape_IR3,onehotencoder__LotShape_Reg,onehotencoder__LandContour_Bnk,onehotencoder__LandContour_HLS,onehotencoder__LandContour_Low,onehotencoder__LandContour_Lvl,onehotencoder__Utilities_AllPub,onehotencoder__Utilities_NoSeWa,onehotencoder__LotConfig_Corner,onehotencoder__LotConfig_CulDSac,onehotencoder__LotConfig_FR2,onehotencoder__LotConfig_FR3,onehotencoder__LotConfig_Inside,onehotencoder__LandSlope_Gtl,onehotencoder__LandSlope_Mod,onehotencoder__LandSlope_Sev,onehotencoder__Neighborhood_Blmngtn,onehotencoder__Neighborhood_Blueste,onehotencoder__Neighborhood_BrDale,onehotencoder__Neighborhood_BrkSide,onehotencoder__Neighborhood_ClearCr,onehotencoder__Neighborhood_CollgCr,onehotencoder__Neighborhood_Crawfor,onehotencoder__Neighborhood_Edwards,onehotencoder__Neighborhood_Gilbert,onehotencoder__Neighborhood_IDOTRR,onehotencoder__Neighborhood_MeadowV,onehotencoder__Neighborhood_Mitchel,onehotencoder__Neighborhood_NAmes,onehotencoder__Neighborhood_NPkVill,onehotencoder__Neighborhood_NWAmes,onehotencoder__Neighborhood_NoRidge,onehotencoder__Neighborhood_NridgHt,onehotencoder__Neighborhood_OldTown,onehotencoder__Neighborhood_SWISU,onehotencoder__Neighborhood_Sawyer,onehotencoder__Neighborhood_SawyerW,onehotencoder__Neighborhood_Somerst,onehotencoder__Neighborhood_StoneBr,onehotencoder__Neighborhood_Timber,onehotencoder__Neighborhood_Veenker,onehotencoder__Condition1_Artery,onehotencoder__Condition1_Feedr,onehotencoder__Condition1_Norm,onehotencoder__Condition1_PosA,onehotencoder__Condition1_PosN,onehotencoder__Condition1_RRAe,onehotencoder__Condition1_RRAn,onehotencoder__Condition1_RRNe,onehotencoder__Condition1_RRNn,onehotencoder__Condition2_Artery,onehotencoder__Condition2_Feedr,onehotencoder__Condition2_Norm,onehotencoder__Condition2_PosA,onehotencoder__Condition2_PosN,onehotencoder__Condition2_RRAe,onehotencoder__Condition2_RRAn,onehotencoder__Condition2_RRNn,onehotencoder__BldgType_1Fam,onehotencoder__BldgType_2fmCon,onehotencoder__BldgType_Duplex,onehotencoder__BldgType_Twnhs,onehotencoder__BldgType_TwnhsE,onehotencoder__HouseStyle_1.5Fin,onehotencoder__HouseStyle_1.5Unf,onehotencoder__HouseStyle_1Story,onehotencoder__HouseStyle_2.5Fin,onehotencoder__HouseStyle_2.5Unf,onehotencoder__HouseStyle_2Story,onehotencoder__HouseStyle_SFoyer,onehotencoder__HouseStyle_SLvl,onehotencoder__OverallQual_1.0,onehotencoder__OverallQual_10.0,onehotencoder__OverallQual_2.0,onehotencoder__OverallQual_3.0,onehotencoder__OverallQual_4.0,onehotencoder__OverallQual_5.0,onehotencoder__OverallQual_6.0,onehotencoder__OverallQual_7.0,onehotencoder__OverallQual_8.0,onehotencoder__OverallQual_9.0,onehotencoder__OverallCond_1.0,onehotencoder__OverallCond_2.0,onehotencoder__OverallCond_3.0,onehotencoder__OverallCond_4.0,onehotencoder__OverallCond_5.0,onehotencoder__OverallCond_6.0,onehotencoder__OverallCond_7.0,onehotencoder__OverallCond_8.0,onehotencoder__OverallCond_9.0,onehotencoder__RoofStyle_Flat,onehotencoder__RoofStyle_Gable,onehotencoder__RoofStyle_Gambrel,onehotencoder__RoofStyle_Hip,onehotencoder__RoofStyle_Mansard,onehotencoder__RoofStyle_Shed,onehotencoder__RoofMatl_ClyTile,onehotencoder__RoofMatl_CompShg,onehotencoder__RoofMatl_Membran,onehote

In [108]:
# Asign columns to feature matrix X and response vector y
X = df.iloc[:, :-1]
y = df.iloc[:,-1]

In [109]:
# Imports
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.compose import make_column_selector as selector

In [110]:
# Train, test set split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=0, 
                                                    stratify=y)

In [111]:
from sklearn.ensemble import RandomForestClassifier


clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", RandomForestClassifier())]
)


clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))
clf

model score: 0.832


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(sparse=False),
                                                  <function <lambda> at 0x7ffdab592b80>)])),
                ('classifier', RandomForestClassifier())])

In [112]:
print(X_train.columns)


Index(['onehotencoder__MSSubClass_120.0', 'onehotencoder__MSSubClass_160.0',
       'onehotencoder__MSSubClass_180.0', 'onehotencoder__MSSubClass_190.0',
       'onehotencoder__MSSubClass_20.0', 'onehotencoder__MSSubClass_30.0',
       'onehotencoder__MSSubClass_40.0', 'onehotencoder__MSSubClass_45.0',
       'onehotencoder__MSSubClass_50.0', 'onehotencoder__MSSubClass_60.0',
       ...
       'remainder__GarageArea', 'remainder__WoodDeckSF',
       'remainder__OpenPorchSF', 'remainder__EnclosedPorch',
       'remainder__3SsnPorch', 'remainder__ScreenPorch', 'remainder__PoolArea',
       'remainder__MiscVal', 'remainder__MoSold', 'remainder__YrSold'],
      dtype='object', length=432)


In [113]:
print(X_test.columns)

Index(['onehotencoder__MSSubClass_120.0', 'onehotencoder__MSSubClass_160.0',
       'onehotencoder__MSSubClass_180.0', 'onehotencoder__MSSubClass_190.0',
       'onehotencoder__MSSubClass_20.0', 'onehotencoder__MSSubClass_30.0',
       'onehotencoder__MSSubClass_40.0', 'onehotencoder__MSSubClass_45.0',
       'onehotencoder__MSSubClass_50.0', 'onehotencoder__MSSubClass_60.0',
       ...
       'remainder__GarageArea', 'remainder__WoodDeckSF',
       'remainder__OpenPorchSF', 'remainder__EnclosedPorch',
       'remainder__3SsnPorch', 'remainder__ScreenPorch', 'remainder__PoolArea',
       'remainder__MiscVal', 'remainder__MoSold', 'remainder__YrSold'],
      dtype='object', length=432)
